### colab으로 할 때만 실행

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!nvidia-smi

In [2]:
import os
os.chdir("/content/drive/MyDrive")

### 설치 필요할때만

In [4]:
!pip install adabelief-tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python --version
!pip install opencv-python
!pip install tensorflow_datasets
!pip install -U scikit-learn
!pip install adabelief-tf

### 기본 설정

In [5]:
from shutil import copy
from collections import defaultdict
import scipy
import numpy as np
import matplotlib.pyplot as plt
import warnings

import random
import cv2
import glob
from PIL import Image, ImageEnhance
import PIL.ImageOps

import torch
import torchvision
from torch import nn


import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.ops.clip_ops import clip_by_global_norm
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
import tensorflow_datasets as tfds


# Theano(th)와 Tensorflow(tf) 모두와 호환이 되는 Keras 모듈을 작성
import keras.backend as K
from keras import regularizers
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, BatchNormalization, Dropout, Flatten, Dense, concatenate
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator # 데이터 전처리
from keras.callbacks import ModelCheckpoint, CSVLogger
#from keras.optimizers import SGD

from keras_preprocessing.image import array_to_img, img_to_array, load_img

from sklearn.model_selection import train_test_split


from deeplab2.model.pixel_encoder import moat
from adabelief_tf import AdaBeliefOptimizer

### 데이터 불러오기

### https://www.tensorflow.org/datasets/catalog/cifar100

In [6]:
# 'cifar100' 대신 'food101' 이나 다른 데이터셋 이름 넣으면 ok
from tensorflow.keras.datasets import cifar10

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

tr_images, val_images, tr_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.15, shuffle=False)
tr1_images, tr2_images, tr1_labels, tr2_labels = train_test_split(tr_images, tr_labels, test_size=0.5, shuffle=False)

nb_train_samples =  tr_images.shape[0]
nb_train1_samples =  tr1_images.shape[0]
nb_train2_samples =  tr2_images.shape[0] 
nb_validation_samples = val_images.shape[0]
nb_test_samples = test_images.shape[0]

print('Train:', tr_images.shape, tr_labels.shape)
print('Train1:', tr1_images.shape, tr1_labels.shape)
print('Train2:', tr2_images.shape, tr2_labels.shape)
print('Valid:', val_images.shape, val_labels.shape)
print('Test:', test_images.shape, test_labels.shape)


170498071/170498071 [==============================] - 2s 0us/step
Train: (42500, 32, 32, 3) (42500, 1)
Train1: (21250, 32, 32, 3) (21250, 1)
Train2: (21250, 32, 32, 3) (21250, 1)
Valid: (7500, 32, 32, 3) (7500, 1)
Test: (10000, 32, 32, 3) (10000, 1)


In [ ]:
print(np.unique(tr_labels, return_counts = True))
print(np.unique(tr1_labels, return_counts = True))
print(np.unique(tr2_labels, return_counts = True))
print(np.unique(val_labels, return_counts = True))
print(np.unique(test_labels, return_counts = True))

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8), array([4239, 4251, 4285, 4228, 4271, 4201, 4262, 4269, 4250, 4244]))
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8), array([2117, 2120, 2162, 2148, 2134, 2040, 2167, 2141, 2139, 2082]))
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8), array([2122, 2131, 2123, 2080, 2137, 2161, 2095, 2128, 2111, 2162]))
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8), array([761, 749, 715, 772, 729, 799, 738, 731, 750, 756]))
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8), array([1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]))


### 데이터 불리기

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.3,        # Shear angle in counter-clockwise direction as radians
    #zoom_range=0.3,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest') 

train1_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.3,        # Shear angle in counter-clockwise direction as radians
    #zoom_range=0.3,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest') 

train2_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.3,        # Shear angle in counter-clockwise direction as radians
    #zoom_range=0.3,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest') 

val_datagen = ImageDataGenerator(
    rescale=1./255,)

test_datagen = ImageDataGenerator(
    rescale=1./255,)

### 모델 생성

In [8]:
moat1 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[32,32,3])
moat1 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")

o1 = moat1.output["stage5"]
r1 = tf.keras.layers.Flatten(name='flatten_layer1')(o1)
r1 = tf.keras.layers.BatchNormalization()(r1)
out1 = tf.keras.layers.Dense(
    units=100, activation='softmax', name='output_layer', kernel_initializer="he_normal")(r1)  

def create_model1():
    return tf.keras.models.Model(moat1.input, out1)

In [9]:
moat2 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[32,32,3])
moat2 = moat._load_moat_pretrained_checkpoint(moat2, path="./model-ckpt-0")

o2 = moat2.output["stage5"]
r2 = tf.keras.layers.Flatten(name='flatten_layer1')(o2)
r2 = tf.keras.layers.BatchNormalization()(r2)
out2 = tf.keras.layers.Dense(
    units=100, activation='softmax', name='output_layer', kernel_initializer="he_normal")(r2)  

def create_model2():
    return tf.keras.models.Model(moat2.input, out2)

In [10]:
batch_size = 100

train_generator = train_datagen.flow(x=tr_images, y=tr_labels, batch_size = batch_size, shuffle=True)
train1_generator = train1_datagen.flow(x=tr1_images, y=tr1_labels, batch_size = batch_size, shuffle=True)
train2_generator = train2_datagen.flow(x=tr2_images, y=tr2_labels, batch_size = batch_size, shuffle=True)

validation_generator = val_datagen.flow(x=val_images, y=val_labels, batch_size = batch_size, shuffle=False)

test_generator = test_datagen.flow(x=test_images, y=test_labels, batch_size = batch_size, shuffle=False)

### Train 1

In [ ]:
batch_size = 100
train1_generator = train1_datagen.flow(x=tr1_images, y=tr1_labels, batch_size = batch_size, shuffle=True)
validation_generator = val_datagen.flow(x=val_images, y=val_labels, batch_size = batch_size, shuffle=False)
warnings.filterwarnings('ignore')

model1 = create_model1()
#model1.load_weights('cifar10_final_disjoint_zoomx_1_1.hdf5')


opt = AdaBeliefOptimizer(learning_rate=1e-3, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14)

model1.compile(optimizer=opt,
          loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())

checkpointer = ModelCheckpoint(filepath='cifar10_final_disjoint_zoomx_1_1.hdf5', verbose=1, monitor ="val_sparse_categorical_accuracy",save_best_only=True)
csv_logger = CSVLogger('cifar10_final_disjoint_zoomx_1_1.log')

history = model1.fit(train1_generator,
                steps_per_epoch = nb_train1_samples // batch_size,      
                validation_data=validation_generator,
                validation_steps=nb_validation_samples // batch_size,  
                epochs=10,                                             
                verbose=1,
                callbacks=[csv_logger, checkpointer])

In [ ]:
batch_size = 200
train1_generator = train1_datagen.flow(x=tr1_images, y=tr1_labels, batch_size = batch_size, shuffle=True)
validation_generator = val_datagen.flow(x=val_images, y=val_labels, batch_size = batch_size, shuffle=False)
warnings.filterwarnings('ignore')

model1 = create_model1()
model1.load_weights('cifar10_final_disjoint_zoomx_1_1.hdf5')


opt = AdaBeliefOptimizer(learning_rate=1e-5, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14)

model1.compile(optimizer=opt,
          loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())

checkpointer = ModelCheckpoint(filepath='cifar10_final_disjoint_zoomx_1_2.hdf5', verbose=1, monitor ="val_sparse_categorical_accuracy",save_best_only=True)
csv_logger = CSVLogger('cifar10_final_disjoint_zoomx_1_2.log')

history = model1.fit(train1_generator,
                steps_per_epoch = nb_train1_samples // batch_size,      
                validation_data=validation_generator,
                validation_steps=nb_validation_samples // batch_size,  
                epochs=300,                                             
                verbose=1,
                callbacks=[csv_logger, checkpointer])

In [ ]:
model1.save_weights('cifar10_final_disjoint_zoomx_1_2_final.hdf5')

In [14]:
moat1.save_weights('moat_cifar10_final_disjoint_zoomx_1_2_final.hdf5')

* train1 : 어큐 96 - 97 / 83 - 84

### Train 2

In [ ]:
batch_size = 200
train2_generator = train2_datagen.flow(x=tr2_images, y=tr2_labels, batch_size = batch_size, shuffle=True)
validation_generator = val_datagen.flow(x=val_images, y=val_labels, batch_size = batch_size, shuffle=False)
warnings.filterwarnings('ignore')

model2 = create_model2()
#model2.load_weights('cifar10_final_disjoint_zoomx_2_1.hdf5')


opt = AdaBeliefOptimizer(learning_rate=1e-3, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14)

model2.compile(optimizer=opt,
          loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())

checkpointer = ModelCheckpoint(filepath='cifar10_final_disjoint_zoomx_2_1.hdf5', verbose=1, monitor ="val_sparse_categorical_accuracy",save_best_only=True)
csv_logger = CSVLogger('cifar10_final_disjoint_zoomx_2_1.log')

history = model2.fit(train2_generator,
                steps_per_epoch = nb_train2_samples // batch_size,      
                validation_data=validation_generator,
                validation_steps=nb_validation_samples // batch_size,  
                epochs=20,                                             
                verbose=1,
                callbacks=[csv_logger, checkpointer])

In [ ]:
batch_size = 200
train2_generator = train2_datagen.flow(x=tr2_images, y=tr2_labels, batch_size = batch_size, shuffle=True)
validation_generator = val_datagen.flow(x=val_images, y=val_labels, batch_size = batch_size, shuffle=False)
warnings.filterwarnings('ignore')

model2 = create_model2()
model2.load_weights('cifar10_final_disjoint_zoomx_2_1.hdf5')


opt = AdaBeliefOptimizer(learning_rate=1e-5, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14)

model2.compile(optimizer=opt,
          loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())

checkpointer = ModelCheckpoint(filepath='cifar10_final_disjoint_zoomx_2_2.hdf5', verbose=1, monitor ="val_sparse_categorical_accuracy",save_best_only=True)
csv_logger = CSVLogger('cifar10_final_disjoint_zoomx_2_2.log')

history = model2.fit(train2_generator,
                steps_per_epoch = nb_train2_samples // batch_size,      
                validation_data=validation_generator,
                validation_steps=nb_validation_samples // batch_size,  
                epochs=300,                                             
                verbose=1,
                callbacks=[csv_logger, checkpointer])

moat2.save_weights('moat_cifar10_final_disjoint_zoomx_2_2_final.hdf5')

* train2 : 어큐 97 / 83 - 84

### 앙상블 V1

In [11]:
moat3 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[32,32,3])
moat4 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[32,32,3])
moat3._name="moat3"
moat4._name="moat4"

for layer in moat4.layers:
    layer._name = layer.name + str("_2")

In [15]:
moat3.load_weights('moat_cifar10_final_disjoint_zoomx_1_2_final.hdf5')
moat4.load_weights('moat_cifar10_final_disjoint_zoomx_2_2_final.hdf5')

moat3.trainable=False
moat4.trainable=False

In [16]:
en_input = Input(shape=(32,32,3))
x = moat3(en_input)
y = moat4(en_input)

b1 = tf.keras.layers.Flatten(name='flatten_layer1')(x["stage5"])
b2 = tf.keras.layers.Flatten(name='flatten_layer2')(y["stage5"])

z= tf.concat([b1, b2],axis=1)

l= Dense(units=256, activation='gelu')(z)
en_out = Dense(units=10, activation='softmax')(l)

In [17]:
def create_model3():
    return tf.keras.models.Model(inputs=en_input, outputs=en_out) 

#moat2 = moat.get_model(
#    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
#moat2 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")

opt2 = AdaBeliefOptimizer(learning_rate=5e-4, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14, clipnorm=1)

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.1)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.



In [ ]:

en_model = create_model3()
en_model.compile(optimizer=opt2,
          loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
#checkpointer = ModelCheckpoint(filepath='ensemble_1.hdf5', verbose=1, save_best_only=True)
#csv_logger = CSVLogger('ensemble_1.hdf5.log')
history = en_model.fit_generator(train_generator, 
#                  steps_per_epoch = nb_train_samples // batch_size,      
                validation_data=validation_generator,
#                  validation_steps=nb_validation_samples // batch_size,  
                epochs=100,                                             
                verbose=1)


<ipython-input-20-f97fd9b4716c>:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = en_model.fit_generator(train_generator,


Epoch 1/100
425/425 [==============================] - 278s 597ms/step - loss: 9.3796 - sparse_categorical_accuracy: 0.7062 - val_loss: 0.7844 - val_sparse_categorical_accuracy: 0.8127
Epoch 2/100
425/425 [==============================] - 245s 578ms/step - loss: 0.6629 - sparse_categorical_accuracy: 0.8427 - val_loss: 1.1293 - val_sparse_categorical_accuracy: 0.7772
Epoch 3/100
425/425 [==============================] - 251s 590ms/step - loss: 0.7128 - sparse_categorical_accuracy: 0.8416 - val_loss: 1.0385 - val_sparse_categorical_accuracy: 0.7887
Epoch 4/100
425/425 [==============================] - 254s 597ms/step - loss: 0.7171 - sparse_categorical_accuracy: 0.8402 - val_loss: 1.2515 - val_sparse_categorical_accuracy: 0.7556
Epoch 5/100
425/425 [==============================] - 247s 582ms/step - loss: 0.6956 - sparse_categorical_accuracy: 0.8428 - val_loss: 0.8904 - val_sparse_categorical_accuracy: 0.7984
Epoch 6/100
425/425 [==============================] - 257s 604ms/step - lo

In [1]:
en_model.save_weights('final_cifar10_en_1.h5')

NameError: ignored